In [ ]:
!pip install accelerate peft bitsandbytes transformers trl tqdm pandas torch transformers

In [3]:
import pandas as pd
import torch
import pandas as pd
from tqdm import tqdm

In [24]:
def format_prompt_test(essay, assignment):
    instructions = (
        "You are my English teacher. Read my essay and assignment."
        "Give me feedback to help me revise. Extract three to five very short excerpts from my essay and give me feedback on those. Keep your feedback very short."
        "List the excerpts and feedback like this:\n"
        "***[excerpt]---[feedback]\n"
        "***[excerpt]---[feedback]\n"
        "***[excerpt]---[feedback]\n"
    )
    return f"### INSTRUCTIONS: {instructions}\n ESSAY: {essay}\n ASSIGNMENT: {assignment}\n ### FEEDBACK:"

In [25]:
# Load dataset
df = pd.read_csv('eval_data_6_30.csv')
df['input'] = df.apply(lambda row: format_prompt_test(row['essay'], row['prompt']), axis=1)

In [6]:
df.head()

,essay,prompt,input
0,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...
1,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...
2,"Dear Principal,\n\nConsidering a policy change...",Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...
3,"2/1/11\n\nDear Mr. Principal,\n\nI believe tha...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...
4,The New school policy\n\nI do not like the new...,Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...


In [14]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("meiflwr/llama2-7b-feedback-ft")
base_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "meiflwr/llama2-7b-feedback-ft")
tokenizer = AutoTokenizer.from_pretrained("meiflwr/llama2-7b-feedback-ft")

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.19s/it]
/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 200, device = 0)
result = pipe(f"{df['input'][0]}")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [38]:
print(result[0]['generated_text'])

### INSTRUCTIONS: You are my English teacher. Read my essay and assignment.Give me feedback to help me revise. Extract three to five very short excerpts from my essay and give me feedback on those. Keep your feedback very short.List the excerpts and feedback like this:
***[excerpt]---[feedback]
***[excerpt]---[feedback]
***[excerpt]---[feedback]

 ESSAY: Dear Principal,

I have heard you are having trouble making a decision on whether students should or shouldn't be required to do community service. I know many people who do community service including myself. From personal experience of community service i can tell you that it can help the way you think and act around certain types of people and ethnic groups. I am not say there is a different way to act around others, although it would be unprofessional to act like you do with your friends around the president of the United States. Therefore all students should be required to perform community service acts.

One of the main reasons c

In [39]:
from transformers import pipeline
def generate(input, model, tokenizer):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 200, device = 0)
    result = pipe(f"{input}")
    res = result[0]['generated_text']
    return res

In [40]:
from tqdm import tqdm
tqdm.pandas()
df['predictions'] = df.progress_apply(lambda row: generate(row['input'], model, tokenizer), axis=1)

100%|██████████| 174/174 [30:17<00:00, 10.45s/it]


In [41]:
import re
def extract_text_after_tag(text, tag):
    pattern = re.compile(f"{re.escape(tag)}(.*)", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    return ""

def extract_feedback_sections(text):
    # Split the text by asterisks to separate sections
    sections = text.split('***')[1:]  # Ignore the part before the first asterisk
    
    result = []
    for section in sections:
        # Use regex to extract the text between * and ---
        excerpt_match = re.search(r'"(.*?)"---', section, re.DOTALL)
        excerpt = excerpt_match.group(1).strip() if excerpt_match else ''

        # Extract the feedback text after --- and before \n
        feedback_match = re.search(r'---(.*?)(\n|$)', section, re.DOTALL)
        #feedback_match = re.search(r'---(.*)', section, re.DOTALL)
        feedback = feedback_match.group(1).strip() if feedback_match else ''

        if len(excerpt) > 0 and len(feedback) > 0:
            result.append({'excerpt': excerpt, 'feedback': feedback})
    
    return result

def expand_pred_df(df, input_column):
    new_rows = []
    for idx, row in df.iterrows():
        input_text = row[input_column]
        sections = extract_feedback_sections(input_text)
        for section in sections:
            new_row = row.to_dict()
            new_row['excerpt'] = section['excerpt']
            new_row['feedback'] = section['feedback']
            new_rows.append(new_row)
    
    expanded_df = pd.DataFrame(new_rows)
    return expanded_df

In [42]:
df['extracted_content'] = df['predictions'].apply(lambda input_text: extract_text_after_tag(input_text, tag = "### FEEDBACK:"))


In [44]:
df.head()

,essay,prompt,input,predictions,extracted_content
0,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""From personal experience of community serv..."
1,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ..."
2,"Dear Principal,\n\nConsidering a policy change...",Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""ethical proposal""---Is this a question?\n*..."
3,"2/1/11\n\nDear Mr. Principal,\n\nI believe tha...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""I believe that students should not be requ..."
4,The New school policy\n\nI do not like the new...,Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""I think that a C average is fine. ""---Good..."


In [45]:
res_df = expand_pred_df(df, 'extracted_content')
res_df.head()

,essay,prompt,input,predictions,extracted_content,excerpt,feedback
0,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""From personal experience of community serv...",From personal experience of community service ...,I would combine this section to make it more c...
1,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""From personal experience of community serv...",One of the main reasons community service shou...,I agree that students have more time and that ...
2,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...","It is helpful to seek advice from others, beca...",I love that you are using personal experiences...
3,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...",A different outlook on things can benefit the ...,Love this idea!
4,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...",Sometimes you could be stuck on something and ...,This could be true! What other times might it ...


In [46]:
res_df.shape

(502, 7)

In [47]:
res_df.to_csv('finetune_res.csv', index=False)

In [2]:
import pandas as pd
# merge with metadata
meta = pd.read_csv('eval_data_6_30_var.csv')
ftres = pd.read_csv('finetune_res.csv')
merged_df = pd.merge(ftres, meta, on=['essay', 'prompt'])

In [3]:
merged_df.head()

,essay,prompt,input,predictions,extracted_content,excerpt,feedback,essayid,grade,type,score,wc,gender,race,ed,ds,ell
0,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""From personal experience of community serv...",From personal experience of community service ...,I would combine this section to make it more c...,1CE6C10B9683,ms,Persuasive,3,363,F,White,0,0,0
1,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""From personal experience of community serv...",One of the main reasons community service shou...,I agree that students have more time and that ...,1CE6C10B9683,ms,Persuasive,3,363,F,White,0,0,0
2,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...","It is helpful to seek advice from others, beca...",I love that you are using personal experiences...,4CB46BC4B97B,ms,Persuasive,4,259,F,White,0,1,0
3,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...",A different outlook on things can benefit the ...,Love this idea!,4CB46BC4B97B,ms,Persuasive,4,259,F,White,0,1,0
4,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""It is helpful to seek advice from others, ...",Sometimes you could be stuck on something and ...,This could be true! What other times might it ...,4CB46BC4B97B,ms,Persuasive,4,259,F,White,0,1,0


In [4]:
merged_df.to_csv('finetune_res.csv')